# Azure AI Agent service - Custom Bing integration

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import datetime
import os
import sys
import time

from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingCustomSearchTool
from azure.ai.agents.models import ListSortOrder
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 16-Jun-2025 15:07:57


## Settings

In [4]:
load_dotenv("azure.env")

True

In [5]:
BING_CUSTOM_CONN_ID = os.environ["BING_CUSTOM_CONN_ID"]

In [6]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

## Initialize Bing Custom Search tool with connection id and instance name

In [7]:
instance_name = "travel"

# Initialize Bing Custom Search tool with connection id and instance name
bing_custom_tool = BingCustomSearchTool(connection_id=BING_CUSTOM_CONN_ID,
                                        instance_name=instance_name)

Exemple of instance definition:

<img src="custom.jpg">

## Agent creation

In [8]:
agent = project_client.create_agent(
    model='gpt-4o',
    name="travel-planner-agent",
    instructions="You are a travel planner dedicated to helping visitors discover top attractions, find the best hotels and restaurants, explore must-see sights, book flight tickets, and navigate public transportation with ease.",
    tools=bing_custom_tool.definitions,
    description="Travel planner"
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_kwWeTJ4gwB5Pl3hU6CWRtUGt


## Examples

In [9]:
prompt="List me the top 10 things to visit in Milan according to tripadvisor"

In [10]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_oNBNYWaLIPrWzKzF7Y8hAOEh
💬 [2] Created message | ID: msg_Equ7A2jSbxW0bC5U6UGfqZhl
🏃 [3] Created run | ID: run_mIcDnpsMepPpM7U9BhUPuGa2
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → List me the top 10 things to visit in Milan according to tripadvisor

💡 → Here are the top 10 attractions in Milan according to Tripadvisor:

1. **Duomo di Milano** - Iconic Gothic cathedral with intricate architecture.
2. **Galleria Vittorio Emanuele II** - Luxury 19th-century shopping arcade.
3. **Il Cenacolo (Leonardo’s Last Supper)** - Home to the famous mural by Leonardo da Vinci.
4. **Castello Sforzesco** - Renaissance fortress housing museums.
5. **I Navigli** - Historic canals area with lively nightlife.
6. **Piazza del Duomo** - A bustling square featuring the Duomo.
7. **Terrazze del Duomo** - Rooftop of the cathedral with stunning city views.
8. **Chiesa di San Maurizi

In [11]:
prompt="What are the best hotels in Milan according to social networks?"

In [12]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_V3FOQxUawqZ960EL07HzyYXY
💬 [2] Created message | ID: msg_aG0AHZPtaB8cbJwobYt4TTEU
🏃 [3] Created run | ID: run_ZtlIbwsB0Kg7WxfeDqr1puGx
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → What are the best hotels in Milan according to social networks?

💡 → Some of the highly-rated hotels in Milan, according to reviews on social networks and traveler opinions include:

1. Mandarin Oriental, Milano - Known for its luxury and exceptional service.
2. Room Mate Giulia - A popular design-focused boutique hotel near the Duomo.
3. Château Monfort - Offering a fairytale-like luxury experience.
4. Park Hyatt Milano - Renowned for its central location and elegant ambiance.
5. Portrait Milano - A recently acclaimed hotel with trendy design and modern amenities【3:0†source】【3:2†source】【3:4†source】.

These hotels consistently receive positive feedback about their central locatio

In [13]:
prompt = "Find some flights from Paris to Milan"

In [14]:
# Step 1: Create a thread
thread = project_client.threads.create()
print(f"🧵 [1] Created thread | ID: {thread.id}")

# Step 2: Send the user's message
message = project_client.messages.create(thread_id=thread.id,
                                         role="user",
                                         content=prompt)
print(f"💬 [2] Created message | ID: {message.id}")

# Step 3: Start a run
run = project_client.runs.create(thread_id=thread.id,
                                 agent_id=agent.id)
print(f"🏃 [3] Created run | ID: {run.id}")

# Step 4: Poll run status
print("⏳ [4] Polling run status...")
while run.status in {"queued", "in_progress", "requires_action"}:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"🔄 Status: {run.status}")

    if run.status == "failed":
        print(f"❌ Run failed | Error: {run.last_error}")
        break

# Step 5: Display agent responses
print("\n🤖 [5] Agent responses:")
messages = project_client.messages.list(thread_id=thread.id,
                                        order=ListSortOrder.ASCENDING)

for msg in messages:
    print("\033[1;34m")  # Blue text for agent messages
    if msg.text_messages:
        for text_message in msg.text_messages:
            print(f"💡 → {text_message.text.value}")
    if msg.url_citation_annotations:
        print("\033[1;32m")  # Green text for citations
        for annotation in msg.url_citation_annotations:
            print(
                f"🔗 URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})"
            )

print("\033[0m")  # Reset color

🧵 [1] Created thread | ID: thread_2FYTEKBzSPKFuEX8v6Qw5GUO
💬 [2] Created message | ID: msg_WARFJc3GBziXBBfPPgHP1YAi
🏃 [3] Created run | ID: run_r3NgUVQFU9JGVMcpk3t29Bj2
⏳ [4] Polling run status...
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: in_progress
🔄 Status: completed

🤖 [5] Agent responses:

💡 → Find some flights from Paris to Milan

💡 → Here are some flight options from Paris (PAR) to Milan (MIL):

1. **Ryanair**:
   - From €19 one-way.
   - Departs from Paris-Beauvais Airport (BVA) to Milan-Bergamo (BGY).
   - Flight duration: Approximately 1 hour 35 minutes.

2. **easyJet**:
   - From €42 one-way.
   - Departs from Paris-Orly (ORY) to Milan-Malpensa (MXP).
   - Duration: About 1 hour 30 minutes.

3. **Air France**:
   - From €94 one-way.
   - Departs Paris-Charles de Gaulle (CDG) to Milan-Linate (LIN).
   - Duration: About 1 hour 25 minutes.

These prices depend on your travel dates and booking timing. For the most accurate and updated information, it is suggested to 

## Post processing

In [15]:
print(
    f"Agent Details:\n"
    f"📛 Name        : {agent.name}\n"
    f"🆔 ID          : {agent.id}\n"
    f"🧠 Model       : {agent.model}\n"
    f"📜 Instructions: {agent.instructions}"
)

Agent Details:
📛 Name        : travel-planner-agent
🆔 ID          : asst_kwWeTJ4gwB5Pl3hU6CWRtUGt
🧠 Model       : gpt-4o
📜 Instructions: You are a travel planner dedicated to helping visitors discover top attractions, find the best hotels and restaurants, explore must-see sights, book flight tickets, and navigate public transportation with ease.


In [16]:
#print("Deleting the agent we have created...")
#project_client.delete_agent(agent.id)
#print(f"Deleted agent, agent ID: {agent.id}")
#print("Done")